# Option Pricing Logic for all Synthetic Data

Make sure to create the required files first in other notebook(s) before evaluating them here

In [1]:
!pip install fbm
!pip install yfinance==0.1.63
!pip install matplotlib

In [2]:
import os
# Set working directory to MarketGenerators folder
# If you are working on LRZ servers, create the folder "MarketGenerators" and then specify something like
path = "/dss/dsshome1/02/YOUR_LRZ_USER_NAME/MarketGenerators"
os.chdir(path)

In [1]:
import numpy as np

from src.evaluate import metrics as m
from src.visualization.plot_option_results import OptionPricingVisualization 
from src.data.make_dataset import DataLoader
import src.helper.utils as utils

In [ ]:
# Define the base directory
base_dir = "numerical_results"
# Define the folders of interest
model_folders = ["GBM", "Kou_Jump_Diffusion"]
# Define input lengths to evaluate
folders_to_include = [
    "n-in=3Y", 
    "n-in=10Y",
    "n-in=100Y", 
    "n-in=1000Y", 
    "n-in=10000Y",
]
relevant_n_Days = [5, 10, 21, 252]
# Define the target subfolders (first all models needed to be run)
target_subfolders = [
    "CWGAN", "GMMN", "RCGAN", "SigCWGAN", "TimeGAN", 
    "CVAE"
]
model_count = 0
# Parameters
S0 = 1.

In [ ]:
for nDays in relevant_n_Days:
    # Calculate Lookback option price every look_back_grid_size days
    lookBackGridSize = 1 if nDays <= 21 else 10
    T = nDays / 252
    t = 0
    # Specify K_grid for output plots here
    K_grid = (
        np.linspace(0.9, 1.1, 33) if nDays <= 10 else 
        np.linspace(0.85, 1.15, 49) if nDays <= 21 else 
        np.linspace(0.6, 1.4, 81)
    )
    # Initialize empty last specification
    last_spec = ""
    # Loop through each model folder
    for model_folder in model_folders:
        print(f"Start evaluating all {model_folder}-based models for {nDays} days maturity...")
        model_path = os.path.join(base_dir, model_folder)

        # Traverse the directory tree and find relevant files
        for root, dirs, files in os.walk(model_path):
            gen_model = os.path.basename(root)

            if gen_model in target_subfolders:
                model_spec = root.split("/")[2]
                nYearsFolderName = root.split("/")[3]
                if nYearsFolderName not in folders_to_include:
                    continue

                # Read the relevant npy files
                generated_file = os.path.join(root, "generated_returns_rescaled.npy")
                input_file = os.path.join(root, "input_returns_unscaled.npy")
                if os.path.exists(generated_file) and (os.path.exists(input_file) or gen_model == "CVAE"):
                    if gen_model == "CVAE":
                        # No input data for CVAE model currently
                        input_file = os.path.join(root[:-4] + "SigCWGAN", "input_returns_unscaled.npy")

                    input_prices_df, generated_prices_df, _ = utils.load_input_and_generated_returns(
                        input_file, generated_file, nDays, T
                    )

                    input_model, model_spec = root.split("/")[1:3]
                    # First folder => New evaluation
                    if model_count == 0:
                        new_input = True
                        closePlots = False
                        # Reset output string
                        summary_output = ""
                        model_desc = "/".join(root.split("/")[1:5])
                        pairs = model_spec.split("_")
                        params = {key: float(value) for key, value in (pair.split("=") for pair in pairs)}
                        print(f"   Evaluating model {model_desc}...")

                        if model_spec != last_spec:
                            # Recalculate important option data as new input model is used
                            print("      New specification")
                            last_spec = model_spec
                            if input_model != "GBM":
                                # Create data for approximate exact price (1,000,000 paths) in the first run
                                # Fix lambda naming:
                                if "lambda" in params:
                                    params["lambda_"] = params.pop("lambda")
                                n_approx = 1000000
                                params["n"] = n_approx
                                params["T"] = T
                                params["n_points"] = nDays + 1
                                params["S0"] = S0
                                # Create large output sample for approximating exact prices (distinct seed)
                                dataloader = DataLoader(method=root.split("/")[1], params=params, seed=314)
                                print(f"      ...Generating {n_approx} paths of {input_model} model...")
                                approx_df = dataloader.create_dataset(output_type="DataFrame")
                                # Use smaller sample for lookback options (runtime reasons)
                                approx_df_lookback = approx_df.iloc[:100000, :]

                            european_engine, asian_engine, lookback_engine = utils.initialize_all_option_engines(
                                input_prices_df, generated_prices_df, T, t=t, S0=S0, approx_exact=(input_model != "GBM")
                            )

                    input_spec = "-".join(root.split("/")[1:5]).replace(".", ",") + f"_nDays={nDays}"

                    # Get respective path metrics
                    if input_model == "GBM":
                        # Exact BS pricing possible in GBM case
                        approx_df = None
                        approx_df_lookback = None
                        european_engine.sigma = params["sigma"]
                        european_engine.r = params["mu"]
                        asian_engine.sigma = params["sigma"]
                        asian_engine.r = params["mu"]
                        lookback_engine.sigma = params["sigma"]
                        lookback_engine.r = params["mu"]
                        exact_label = "Theoretical "
                        with utils.Capturing(summary_output) as summary_output:
                            print(gen_model + " " + input_spec)
                            m.print_basic_gbm_metrics(
                                n_periods=T, 
                                annualization_factor=252, 
                                ground_paths_df=input_prices_df, 
                                recovered_paths_df=generated_prices_df, 
                                exp_stdev=european_engine.sigma,  
                                mu=european_engine.r,  
                                return_threshold=0.03
                            )
                    else:
                        # Approximate pricing for non-GBM models
                        exact_label = "Approx. "
                        with utils.Capturing(summary_output) as summary_output:
                            # Get respective path metrics and save drift for MC
                            print(gen_model + " " + input_spec)
                            european_engine.r = m.print_basic_non_gbm_metrics( 
                                n_periods=T,
                                annualization_factor=252, 
                                ground_paths_df=input_prices_df, 
                                recovered_paths_df=generated_prices_df, 
                                approx_df=approx_df,
                                return_threshold=0.03
                            )
                            asian_engine.r = european_engine.r
                            lookback_engine.r = european_engine.r

                    print(f"         Calculating option prices for {gen_model} paths")
                    # Calculate all values (option prices & deviations) for different strike prices (K or T values)
                    european_engine.gen_paths_df = generated_prices_df
                    european_engine.ground_paths_df = input_prices_df
                    european_engine.calc_all_K(K_grid, approx_df=approx_df, recalculate_input=new_input)
                    asian_engine.gen_paths_df = generated_prices_df
                    asian_engine.ground_paths_df = input_prices_df
                    asian_engine.calc_all_K(K_grid, approx_df=approx_df, recalculate_input=new_input)
                    # Use less data for runtime reasons for approx_df for lookback options logic
                    lookback_engine.gen_paths_df = generated_prices_df
                    lookback_engine.ground_paths_df = input_prices_df
                    lookback_engine.calc_all_T(grid_size=lookBackGridSize, approx_df=approx_df_lookback, recalculate_input=new_input)
                    new_input = False
                    # Plot all types of plots
                    if model_count == 0:
                        # Initialize new plotter if new setup is present
                        european_plotter = OptionPricingVisualization(european_engine, exact_label=exact_label, file_name=input_spec)
                        asian_plotter = OptionPricingVisualization(asian_engine, exact_label=exact_label, file_name=input_spec)
                        lookback_plotter = OptionPricingVisualization(lookback_engine, exact_label=exact_label, file_name=input_spec)

                    # Update pricing engine for respective model
                    european_plotter.pe = european_engine
                    asian_plotter.pe = asian_engine
                    lookback_plotter.pe = lookback_engine
                    model_count += 1

                    # Save all results to files
                    if model_count == len(target_subfolders):
                        # Specify file location
                        relevant_dir = "/".join(root.split("/")[:-1])
                        print(f"            Writing summary file at {relevant_dir}.")
                        with open(f"{relevant_dir}/basic_metrics_nDays={nDays}.txt", "w") as text_file:
                            text_file.write("\n".join(summary_output))
                        closePlots = True
                        # Reset model counter
                        model_count = 0

                    # Plot option results
                    european_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    ylimits_abs_dev = european_plotter.set_y_limits_dev_abs(nDays, model_folder)
                    european_plotter.plot_option_price_deviation(close=closePlots, label=gen_model, zoom_ylimits=ylimits_abs_dev)
                    european_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0, 55))

                    asian_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    ylimits_abs_dev = asian_plotter.set_y_limits_dev_abs(nDays, model_folder)
                    asian_plotter.plot_option_price_deviation(close=closePlots, label=gen_model, zoom_ylimits=ylimits_abs_dev)
                    asian_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0, 55))

                    lookback_plotter.plot_option_prices(close=closePlots, label=gen_model)
                    ylimits_abs_dev = lookback_plotter.set_y_limits_dev_abs(nDays, model_folder)
                    lookback_plotter.plot_option_price_deviation(close=closePlots, label=gen_model, zoom_ylimits=ylimits_abs_dev)
                    lookback_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0, 55))
                else:
                    print("No file found yet.")

print("Done.")